## Concatenating all attention distributions 

We extract the attention distribution directly from a model that has not been pretrained for bias detection! We then using windowing to extract a window of attention scores around the biased word. The windowing we use is around the actual biased word - not the predicted word.

In [2]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist, window_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
params = Params.read_params("linear-params-window.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
reducer = concat


In [5]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dev_dataset = ExperimentDataset.init_dataset(params.dataset)

04/08/2020 06:48:52 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
100it [00:00, 3093.05it/s]


In [6]:
import pickle 
train_dataset = pickle.load(open(params.dataset["weakly_labeled_data"], "rb"))

### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [7]:
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True, from_pretrained=False)

04/08/2020 06:49:02 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/08/2020 06:49:03 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/08/2020 06:49:03 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmplfzd3gc2
04/08/2020 06:49:09 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with default HuggingFace weights.
Succesfully loaded in attention experiment!


In [8]:
attention_dataloader_dev = dev_dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size'])  
attention_dataloader_train = train_dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size'])

```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [9]:
attention_scores_dev = attention_experiment.extract_attention_scores(attention_dataloader_dev)

In [10]:
# Saving out attention weights from the train dataset to facilitate future runs
import os 
attention_weights_file = "model_weights/attention_scores_train.pkl"
if os.path.exists(attention_weights_file):
    print("Loading in existing train attention weights")
    attention_scores_train = pickle.load(open(attention_weights_file, "rb"))
else:
    print("Generating new training attention weights ")
    if (not os.path.isdir("model_weights")):
            os.mkdir("model_weights")
    attention_scores_train = attention_experiment.extract_attention_scores(attention_dataloader_train)
    pickle.dump(attention_scores_train, open(attention_weights_file, "wb+"))

Loading in existing train attention weights


Using the actual occurence of bias to window attention distributions around

In [13]:
from src.utils.data_utils import get_bias_indices

In [14]:
bias_indices_train = get_bias_indices(train_dataset.get_val('pre_tok_label_ids'))

In [20]:
bias_indices_dev = get_bias_indices(dev_dataset.get_val('pre_tok_label_ids'))

In [21]:
attention_scores_indexed_train = return_idx_attention_dist(attention_scores_train, bias_indices_train)
reduced_attention_train = reduce_attention_dist(attention_scores_indexed_train, params.intermediary_task["attention"])
windowed_dist_train = window_attention_dist(reduced_attention_train, bias_indices_train, window_size=7, num_concat=len(params.intermediary_task["attention"]["layers"]))
attention_dist_train = windowed_dist_train

In [22]:
windowed_dist_train.shape

torch.Size([52275, 180])

In [23]:
train_dataset.add_data(attention_dist_train, "attention_dist")
train_dataset.shuffle_data()
assert(attention_dist_train.shape[1] == params.final_task['input_dim'])

In [24]:
attention_scores_indexed_dev = return_idx_attention_dist(attention_scores_dev, bias_indices_dev)
reduced_attention_dev = reduce_attention_dist(attention_scores_indexed_dev, params.intermediary_task["attention"])
windowed_dist_dev = window_attention_dist(reduced_attention_dev, bias_indices_dev, window_size=7, num_concat=len(params.intermediary_task["attention"]["layers"]))
attention_dist_dev = windowed_dist_dev

In [25]:
dev_dataset.add_data(attention_dist_dev, "attention_dist")
dev_dataset.add_data(dev_dataset.get_val('bias_label'),'weak_bias_label')
dev_dataset.shuffle_data()
assert(attention_dist_dev.shape[1] == params.final_task['input_dim'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [26]:
params = Params.read_params("linear-params-window.json")

In [27]:
train_dataloader = train_dataset.return_dataloader(batch_size=params.final_task['training_params']['batch_size'])
dev_dataloader = dev_dataset.return_dataloader(batch_size=32)

In [33]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [34]:
losses, evals = classification_experiment.train_model(train_dataloader, dev_dataloader, input_key="attention_dist", label_key="weak_bias_label")

In [35]:
from src.utils.classification_utils import average_data

In [36]:
avg_evaluations = [average_data(epoch_evaluations) for epoch_evaluations in evals]

In [37]:
avg_evaluations

[{'num_examples': 81,
  'accuracy': 0.5802469135802468,
  'auc': 0.6617350593958782,
  'precision': 0.5978333509197706,
  'recall': 0.8729333622121147,
  'f1': 0.7092102684695277},
 {'num_examples': 81,
  'accuracy': 0.6296296296296295,
  'auc': 0.7064937805678546,
  'precision': 0.6420110932188227,
  'recall': 0.8290376146126635,
  'f1': 0.7231492787048341},
 {'num_examples': 81,
  'accuracy': 0.6790123456790123,
  'auc': 0.7398097534549777,
  'precision': 0.68013793327803,
  'recall': 0.849830337159772,
  'f1': 0.7551304662415774},
 {'num_examples': 81,
  'accuracy': 0.7037037037037037,
  'auc': 0.7619547083484705,
  'precision': 0.705456683234461,
  'recall': 0.8509854884123891,
  'f1': 0.7701804368471035},
 {'num_examples': 81,
  'accuracy': 0.7037037037037037,
  'auc': 0.7779429796973656,
  'precision': 0.705456683234461,
  'recall': 0.8509854884123891,
  'f1': 0.7701804368471035},
 {'num_examples': 81,
  'accuracy': 0.7037037037037037,
  'auc': 0.7795106849687746,
  'precision': 

In [24]:
# classification_experiment.save_model_weights("linear-windowed-attention-actual.weights")